# Utils

In [11]:
import numpy as np
import plotly.graph_objects as go


def plot_failure(data):
    # Extract x, y, and intensity values
    x = sorted({key for subdict in data.values() for key in subdict.keys()})
    y = sorted(data.keys())
    z = np.zeros((len(y), len(x)))

    # Populate the intensity grid
    for yi, y_key in enumerate(y):
        for xi, x_key in enumerate(x):
            z[yi, xi] = data[y_key].get(x_key, 0)

    # Define custom colorscale
    colorscale = [
        [0, "green"],
        [0.01, "white"],
        [0.33, "orange"],
        [0.66, "orangered"],
        [1, "red"],
    ]

    # Create heatmap
    fig = go.Figure(
        data=go.Heatmap(
            z=z,
            x=x,
            y=y,
            colorscale=colorscale,
            colorbar=dict(title="Failed"),
            texttemplate="%{z:.2f}"
        )
    )

    # Style & Layout
    fig.update_layout(
        title="Execution failed across VPREC formats",
        xaxis=dict(title="Precision", tickvals=x, ticks="outside"),
        yaxis=dict(title="Range", tickvals=y, ticks="outside"),
    )

    fig.show()

In [119]:
import numpy as np
import plotly.graph_objects as go


def plot_metric_diff(failures: dict[dict[int]], metrics: dict[float], *, n_subjects: int, metric_name: str, exp_metrics: dict[float]=None,):
    # Extract x, y, and intensity values
    failures = failures.copy()
    x = sorted({key for subdict in failures.values() for key in subdict.keys()})
    y = sorted(failures.keys())
    z = np.zeros((len(y), len(x)))

    # Populate the intensity grid
    for yi, y_key in enumerate(y):
        for xi, x_key in enumerate(x):
            z[yi, xi] = metrics.get(f"r{y_key}-p{x_key}", np.nan)

    # Difference between binary64
    delta = z - metrics["binary64"]
    m = np.nanmin(delta)
    M = np.nanmax(delta)
    norm_pivot = (0 - m) / (M - m)

    
    # Convert NaN to 1 to display with colorscale
    masked_failed = np.where(np.isnan(delta), 1, np.nan)
    delta = np.where(np.isnan(delta), np.nan, delta)


    if exp_metrics:
        std_delta = np.zeros((len(y), len(x)))
        for yi, y_key in enumerate(y):
            for xi, x_key in enumerate(x):
                std_delta[yi, xi] = np.nanstd(
                    np.asarray(exp_metrics.get(f"r{y_key}-p{x_key}", np.nan)) - np.asarray(exp_metrics["binary64"])
                )

        formatter = np.vectorize(lambda d, s: f"{d:.3f} ± {s:.3f}")
        formatted = formatter(delta, std_delta)
    else:
        formatter = np.vectorize(lambda d: f"{d:.3f}")
        formatted = formatter(delta)


    text_labels = np.where(delta < 1, formatted, f"⚠ Err: {n_subjects}/{n_subjects}")

    # Define custom colorscale
    colorscale = [
        [0, "green"],
        [norm_pivot, "white"],
        [1, "red"],
    ]

    # Create heatmap
    normal_trace = go.Heatmap(
        z=delta,
        x=x,
        y=y,
        colorscale=colorscale,
        zmin=m,
        zmax=M,
        colorbar=dict(title="Metric"),
        text=text_labels,
        texttemplate="%{text}",
    )
    failed_trace = go.Heatmap(
        z=masked_failed,
        x=x,
        y=y,
        colorscale=[[0, "grey"], [1, "grey"]],
        showscale=False,
    )
    fig = go.Figure(data=[failed_trace, normal_trace])

    # Annotate failures
    for i in y:
        for j in x:
            cell_value = failures[i][j]

            if cell_value > 0 and cell_value < n_subjects:
                fig.add_annotation(
                x=j,
                y=i - 0.4,         # Place the star at the top edge of the cell
                text=f"⚠ Err: {cell_value}/{n_subjects}",          # Star symbol
                showarrow=False,
                font=dict(color="black"),
                xanchor="center",
                yanchor="bottom"   # Align the bottom of the text with the coordinate
            )

    # Style & Layout
    fig.update_layout(
        title=f"Metric's difference with binary64 - ({metric_name.capitalize()})",
        xaxis=dict(title="Precision", tickvals=x, ticks="outside"),
        yaxis=dict(title="Range", tickvals=y, ticks="outside"),
    )

    fig.show()

In [127]:
import numpy as np
import plotly.graph_objects as go


def plot_metric_rel(failures: dict[dict[int]], metrics: dict[float], *, n_subjects: int, metric_name: str, exp_metrics: dict[float]=None,):
    # Extract x, y, and intensity values
    failures = failures.copy()
    x = sorted({key for subdict in failures.values() for key in subdict.keys()})
    y = sorted(failures.keys())
    z = np.zeros((len(y), len(x)))

    # Populate the intensity grid
    for yi, y_key in enumerate(y):
        for xi, x_key in enumerate(x):
            z[yi, xi] = metrics.get(f"r{y_key}-p{x_key}", np.nan)

    # Difference between binary64
    # delta = np.abs(z - metrics["binary64"]) / np.abs(metrics["binary64"])
    delta = (z - metrics["binary64"]) / np.abs(metrics["binary64"])
    m = np.nanmin(delta)
    M = np.nanmax(delta)
    norm_pivot = (0 - m) / (M - m)

    # Convert NaN to 1 to display with colorscale
    masked_failed = np.where(np.isnan(delta), 1, np.nan)
    delta = np.where(np.isnan(delta), np.nan, delta)


    if exp_metrics:
        std_delta = np.zeros((len(y), len(x)))
        for yi, y_key in enumerate(y):
            for xi, x_key in enumerate(x):
                std_delta[yi, xi] = np.nanstd(np.abs(
                    np.asarray(exp_metrics.get(f"r{y_key}-p{x_key}", np.nan)) - np.asarray(exp_metrics["binary64"])
                ) / np.abs(exp_metrics["binary64"]))

        formatter = np.vectorize(lambda d, s: f"{d:.3f} ± {s:.3f}")
        formatted = formatter(delta, std_delta)
    else:
        formatter = np.vectorize(lambda d: f"{d:.3f}")
        formatted = formatter(delta)

    text_labels = np.where(delta < 1, formatted, f"⚠ Err: {n_subjects}/{n_subjects}")

    # Define custom colorscale
    # colorscale = [
    #     [0, "white"],
    #     [1, "red"],
    # ]
    colorscale = [
        [0, "green"],
        [norm_pivot, "white"],
        [1, "red"],
    ]

    # Create heatmap
    normal_trace = go.Heatmap(
        z=delta,
        x=x,
        y=y,
        colorscale=colorscale,
        zmin=m,
        zmax=M,
        colorbar=dict(title="Metric"),
        text=text_labels,
        texttemplate="%{text}",
    )
    failed_trace = go.Heatmap(
        z=masked_failed,
        x=x,
        y=y,
        colorscale=[[0, "grey"], [1, "grey"]],
        showscale=False,
    )
    fig = go.Figure(data=[failed_trace, normal_trace])

    # Annotate failures
    for i in y:
        for j in x:
            cell_value = failures[i][j]

            if cell_value > 0 and cell_value < n_subjects:
                fig.add_annotation(
                x=j,
                y=i - 0.4,         # Place the star at the top edge of the cell
                text=f"⚠ Err: {cell_value}/{n_subjects}",          # Star symbol
                showarrow=False,
                font=dict(color="black"),
                xanchor="center",
                yanchor="bottom"   # Align the bottom of the text with the coordinate
            )

    # Style & Layout
    fig.update_layout(
        title=f"Metric's relative error with binary64 - ({metric_name.capitalize()})",
        xaxis=dict(title="Precision", tickvals=x, ticks="outside"),
        yaxis=dict(title="Range", tickvals=y, ticks="outside"),
    )

    fig.show()

In [2]:
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Avoid bokeh dependency, by hardcoding the colorblind8 palette
Colorblind8 = (
    "#0072B2",
    "#E69F00",
    "#F0E442",
    "#009E73",
    "#56B4E9",
    "#D55E00",
    "#CC79A7",
    "#000000",
)


def _make_agg_figure(
    data,
    *,
    out_dir: Path,
    y_value: str = None,
    y_log: bool = False,
    show: bool = False,
    exp_ids: list[str] = None,
):
    # Unique values for levels and exp_ids to determine grid size
    levels = data["level"].unique()
    stages = data["stage"].unique()
    if not exp_ids:
        exp_ids = data["exp_id"].unique()

    # Creating the subplot grid with independent x-axes
    fig = make_subplots(
        rows=len(stages),
        cols=len(levels),
        shared_xaxes=False,
        shared_yaxes=True,
    )

    # Loop through each level and exp_id to add traces to the respective subplot
    for l, level in enumerate(levels, 1):
        for s, stage in enumerate(stages, 1):
            for e, exp_id in enumerate(exp_ids):
                for i, subject in enumerate(data["subject"].unique()):
                    subset = data[
                        (data["level"] == level)
                        & (data["stage"] == stage)
                        & (data["exp_id"] == exp_id)
                        & (data["subject"] == subject)
                    ]

                    # Line color
                    color = px.colors.qualitative.Dark24[
                        e % len(px.colors.qualitative.Dark24)
                    ]

                    if exp_id == "binary64":
                        color = "black"
                        dash = "dot"
                    else:
                        color = f"rgba{px.colors.hex_to_rgb(color) + (0.4,)}"
                        dash = "solid"


                    fig.add_trace(
                        go.Scatter(
                            x=subset["iterations"],
                            y=subset[y_value],
                            name=exp_id,
                            mode="lines",
                            fillcolor=color,
                            line=dict(color=color, dash=dash),
                            legendgroup=exp_id,
                            showlegend=(
                                True if (l == 1 and s == 1 and i == 0) else False
                            ),
                            hovertemplate=f"Subject: {subject}<br>Iteration: %{{x}}<br>{y_value}: %{{y}}",
                        ),
                        row=s,
                        col=l,
                    )

    for i, sigma in enumerate(list(range(len(levels)))[::-1], 1):
        fig.add_annotation(
            text=f"Sigma: {sigma}",
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=1.05,
            xanchor="center",
            yanchor="bottom",
            row=1,
            col=i,
            showarrow=False,
            font=dict(size=14, color="black"),
        )

    for i, stage in enumerate(["Rigid", "Affine", "SyN"], 1):
        fig.add_annotation(
            text=stage,
            xref="x domain",
            yref="y domain",
            x=1.05,
            y=0.4,
            xanchor="center",
            yanchor="bottom",
            textangle=90,
            row=i,
            col=len(levels),
            showarrow=False,
            font=dict(size=14, color="black"),
        )

    fig.update_xaxes(title_text="Iterations", row=3)
    fig.update_yaxes(title_text=y_value.replace("_", " ").capitalize(), col=1)
    if y_log:
        fig.update_yaxes(type="log", exponentformat="e")
    fig.update_layout(
        title=f"All subjects",
        showlegend=True,
        legend_title="Experiment ID",
        width=1920,
        height=1080,
    )

    if show:
        fig.show()
    out_dir.joinpath(y_value).mkdir(exist_ok=True)
    fig.write_html(out_dir / y_value / f"all_subjects.html")
    fig.write_image(out_dir / y_value / f"all_subjects.png")

# Data processing

In [3]:
from pathlib import Path
import re

DEBUG = True
N_SUBJECT = 20

FILENAME_PATTERN = re.compile(
    r"r(?P<range>\d+)-p(?P<precision>\d+)-(?P<task_id>\d+)-(?P<array_id>\d+)\.out"
)

## Check failed execution

In [4]:
from typing import Iterable


def get_failed_exec(logs: Path | Iterable[Path]) -> int:
    if isinstance(logs, Path):
        logs = [logs]

    failed = list()
    for log in logs:
        with open(log) as f:
            if "ITK ERROR" in f.read():
                failed.append(log)
    return failed

In [5]:
# Group logs by task_id
import re
from collections import defaultdict


# logs = list(Path("log").glob(f"r*-p*-*.out"))
logs = list(Path("log-all-instrumented").glob(f"r*-p*-*.out"))

experiements = defaultdict(list)
for log in logs:
    m = FILENAME_PATTERN.match(log.name)
    if not m:
        print(log)
        raise ValueError("Invalid log file name")
    experiements[m.group("task_id")].append(log)

In [6]:
defaultdict(dict)

defaultdict(dict, {})

In [7]:
# Count failed execution for each task_id
n_failed = defaultdict(dict)
for task_id, logs in experiements.items():
    failed = get_failed_exec(logs)

    range_ = int(FILENAME_PATTERN.match(logs[0].name).group("range"))
    precision_ = int(FILENAME_PATTERN.match(logs[0].name).group("precision"))
    n_failed[range_][precision_] = len(failed)

    print(f"Task {task_id}: {len(failed)} failed")
    if DEBUG and len(failed) != len(logs):
        for log in failed:
            print(log, end="\n\n")

# Manually add failure for range 6
n_failed[6] = {r: N_SUBJECT for r in range(7, 24)}

Task 157950: 20 failed
Task 157953: 20 failed
Task 157954: 20 failed
Task 157952: 20 failed
Task 157951: 20 failed
Task 157955: 0 failed
Task 157956: 0 failed
Task 157957: 1 failed
log-all-instrumented/r7-p14-157957-9.out

Task 157958: 1 failed
log-all-instrumented/r7-p15-157958-14.out

Task 157959: 0 failed
Task 157960: 0 failed
Task 157961: 0 failed
Task 157981: 0 failed
Task 157982: 0 failed
Task 157983: 0 failed
Task 157984: 0 failed
Task 157985: 0 failed
Task 157986: 20 failed
Task 157987: 20 failed
Task 157988: 20 failed
Task 157989: 20 failed
Task 157990: 20 failed
Task 157991: 0 failed
Task 157992: 0 failed
Task 157993: 1 failed
log-all-instrumented/r8-p14-157993-9.out

Task 157994: 1 failed
log-all-instrumented/r8-p15-157994-14.out

Task 157995: 0 failed
Task 157996: 0 failed
Task 157997: 0 failed
Task 157998: 0 failed
Task 157999: 0 failed
Task 158000: 0 failed
Task 158001: 0 failed
Task 158002: 0 failed


## QA using metric value

In [8]:
import pandas as pd


p_lvl_header = r"DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST|  Elapsed time"

logs = Path("log-all-instrumented")

dfs = list()
for array_id in range(1, N_SUBJECT + 1):
    filenames = list(logs.glob(f"r*-p*-*-{array_id}.out")) + list(
        logs.glob(f"antsRegistration-*-{array_id}.out")
    )
    for filename in filenames:
        if filename.name.startswith("antsRegistration-"):
            exp_id = "binary64"
        else:
            exp_id = "-".join(filename.name.split("-")[:2])

        txt = filename.read_text()

        # Extract subject_id from log
        subject_id = re.search(r"SUBJECT_ID: (?P<subject_id>.*)", txt).group(
            "subject_id"
        )

        # Filter out the header from the stages
        all_data = [x for i, x in enumerate(re.split(p_lvl_header, txt)) if i % 5 != 0]

        # 3 stages with 4 levels of resolution each
        for stage in range(1, 4):
            for i, level in enumerate(all_data[4 * (stage - 1) : 4 * stage]):
                table = defaultdict(list)
                for row in re.split(r"\n", level.strip("XX").strip()):
                    # Skip invalid rows
                    if not ("2DIAGNOSTIC" in row or "1DIAGNOSTIC" in row):
                        continue

                    # Raise exception if the row is not as expected
                    try:
                        cols = row.split(",")
                        table["iterations"].append(cols[1].strip())
                        table["metric"].append(cols[2].strip())
                        table["convergence_value"].append(cols[3].strip())
                        table["total_time"].append(cols[4].strip())
                        table["since_last"].append(cols[5].strip())
                    except Exception as e:
                        print(cols)
                        raise e

                dfs.append(
                    pd.DataFrame(
                        data={
                            "subject": subject_id,
                            "array_id": array_id,
                            "exp_id": exp_id,
                            "stage": stage,
                            "level": i + 1,
                            "iterations": table["iterations"],
                            "metric": table["metric"],
                            "convergence_value": table["convergence_value"],
                            "total_time": table["total_time"],
                            "since_last": table["since_last"],
                        }
                    )
                )

df = pd.concat(dfs, ignore_index=True)
df = df.astype(
    {
        "stage": int,
        "level": int,
        "iterations": int,
        "metric": float,
        "convergence_value": float,
        "total_time": float,
        "since_last": float,
    }
)
rv = (
    df.groupby(["subject", "array_id", "exp_id", "stage", "level", "iterations"])
    .agg({"metric": "mean", "convergence_value": "mean", "total_time": "mean"})
    .reset_index()
)

In [9]:
figure_dir = Path("figures", "VPREC-exploration")
figure_dir.mkdir(exist_ok=True)

exp_ids = [
    "r7-p12",
    "r7-p13",
    "r7-p14",
    "r7-p15",
    "r7-p16",
    "r7-p17",
    "r7-p18",
    "r7-p19",
    "r7-p20",
    "r7-p21",
    "r7-p22",
    "r7-p23",
    "r8-p12",
    "r8-p13",
    "r8-p14",
    "r8-p15",
    "r8-p16",
    "r8-p17",
    "r8-p18",
    "r8-p19",
    "r8-p20",
    "r8-p21",
    "r8-p22",
    "r8-p23",
    "binary64"
]
# exp_ids = ['r7-p12', 'r7-p16', 'r7-p20', 'r7-p23', 'r8-p12', 'r8-p16', 'r8-p20', 'r8-p23']

# _make_agg_figure(rv, out_dir=figure_dir, y_value="metric", show=True, exp_ids=exp_ids)

## Refined region plot

In [10]:
import numpy as np

max_iterations = rv[(rv["stage"]==3) & (rv["level"] == 4)].groupby(["subject", "exp_id"])["iterations"].max().to_dict()
subjects = rv["subject"].unique()
exp_ids = rv["exp_id"].unique()
exp_metrics = defaultdict(list)
for subject in subjects:
    for exp_id in exp_ids:
        try:
            _metric = rv[
                (rv["subject"] == subject)
                & (rv["exp_id"] == exp_id)
                & (rv["stage"] == 3)
                & (rv["level"] == 4)
                & (rv["iterations"] == max_iterations.get((subject, exp_id), 0))
                ]["metric"].values[0]
        except IndexError as e:
            if max_iterations.get((subject, exp_id), 0) == 0:
                _metric = np.nan
            else:
                raise e
        finally:
            exp_metrics[exp_id].append(_metric)


In [ ]:
worst_metrics = {k: np.nanmax(v) for k, v in exp_metrics.items()}
mean_metrics = {k: np.nanmean(v) for k, v in exp_metrics.items()}
n_subjects = rv["subject"].nunique()

/tmp/ipykernel_3450309/2442978351.py:1: RuntimeWarning: All-NaN axis encountered
  worst_metrics = {k: np.nanmax(v) for k, v in exp_metrics.items()}
/tmp/ipykernel_3450309/2442978351.py:2: RuntimeWarning: Mean of empty slice
  mean_metrics = {k: np.nanmean(v) for k, v in exp_metrics.items()}
/home/mathdugre/papers/fast-interpolate/.venv/fast-interpolate/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [121]:
plot_metric_diff(n_failed, worst_metrics, n_subjects=n_subjects, metric_name="Worst subject")
plot_metric_diff(n_failed, mean_metrics, n_subjects=n_subjects, exp_metrics=exp_metrics, metric_name="Mean across subjects")

In [128]:
plot_metric_rel(n_failed, worst_metrics, n_subjects=n_subjects, metric_name="Worst subject")
plot_metric_rel(n_failed, mean_metrics, n_subjects=n_subjects, exp_metrics=exp_metrics, metric_name="Mean across subjects")